In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from notebookutils import mssparkutils
import pandas as pd

print(f"Versión de Spark: {spark.version}")

StatementMeta(, d9d065e3-161e-424b-9833-db7fda3a5810, 4, Finished, Available, Finished)

Versión de Spark: 3.5.1.5.4.20250519.1


In [ ]:
spark =

In [ ]:
# Crear estructura de esquemas en el lakehouse para almacenar nuestras tablas procesadas
spark = SparkSession.builder.getOrCreate()
try:
    #creando carpetas para datos procesados, esquema medallion. Premisa: Capa Bronze existente, Pipeline Copy Data
    spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
    spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
    spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
    print("Esquemas creados")
except Exception as e:
    print(f"Error al crear esquemas: {str(e)}")

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 25, Finished, Available, Finished)

Error al crear esquemas: An error occurred while calling o348.sql.
: java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
	at com.microsoft.azure.trident.spark.TridentCoreProxy.failCreateDbIfTrident(TridentCoreProxy.java:300)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createDatabase(SessionCatalog.scala:317)
	at org.apache.spark.sql.execution.datasources.v2.V2SessionCatalog.createNamespace(V2SessionCatalog.scala:327)
	at org.apache.spark.sql.connector.catalog.DelegatingCatalogExtension.createNamespace(DelegatingCatalogExtension.java:163)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:47)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from notebookutils import mssparkutils as mu
import pandas as pd

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 20, Finished, Available, Finished)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Ruta raíz del Lakehouse por defecto en Fabric
lakehouse_base_path = "/lakehouse/default"

# Nombres de esquemas/capas
layers = ["bronze", "silver", "gold"]

try:
    for layer in layers:
        # Crear base de datos en Spark (esquema lógico)
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {layer}")
        
        # Crear carpeta física en el Lakehouse si no existe
        path = f"{lakehouse_base_path}/{layer}"
        dbutils.fs.mkdirs(path)
        
        print(f"Esquema y carpeta '{layer}' creados o ya existentes.")
except Exception as e:
    print(f"❌ Error: {str(e)}")

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 22, Finished, Available, Finished)

❌ Error: An error occurred while calling o348.sql.
: java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
	at com.microsoft.azure.trident.spark.TridentCoreProxy.failCreateDbIfTrident(TridentCoreProxy.java:300)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createDatabase(SessionCatalog.scala:317)
	at org.apache.spark.sql.execution.datasources.v2.V2SessionCatalog.createNamespace(V2SessionCatalog.scala:327)
	at org.apache.spark.sql.connector.catalog.DelegatingCatalogExtension.createNamespace(DelegatingCatalogExtension.java:163)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:47)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.spark.sql.execut

In [ ]:
try:
    mssparkutils.fs.mkdirs("Files/bronze")
    mssparkutils.fs.mkdirs("Files/silver")
    mssparkutils.fs.mkdirs("Files/gold")
    mssparkutils.fs.mkdirs("Files/processed")
    print("✅ Estructura de carpetas creada correctamente")
except Exception as e:
    print(f"Error al crear carpetas: {str(e)}")
    


StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 8, Finished, Available, Finished)

✅ Estructura de carpetas creada correctamente


In [ ]:
schema_brokers = StructType([
    StructField("BrokerID",IntegerType(),False),
    StructField("BrokerName",StringType(),False),
    StructField("Region",StringType(),False),
    StructField("Email",StringType(),False)

])

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 9, Finished, Available, Finished)

In [ ]:
try:
    df_brokers =  spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema_brokers) \
    .load("Files/raw/brokers.csv")

except Exception as e:
    print(f"Error al leer archivos: {str(e)}")

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 10, Finished, Available, Finished)

In [ ]:
df_brokers_bronze = df_brokers.withColumn("_Load_date",F.current_timestamp()) \
.withColumn("_user",F.lit("admin"))

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 11, Finished, Available, Finished)

In [ ]:
try:
    df_brokers_bronze.write.format("delta").mode("overwrite").save("Files/bronze/brokers_bronze")

except Exception as e:
    print(f"Error al leer archivos: {str(e)}")

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 12, Finished, Available, Finished)

In [ ]:
df_brokers_bronze.createOrReplaceTempView("bronze_brokers")
df_brokers_bronze.write.format("delta").mode("overwrite").saveAsTable("bronze.brokers")

StatementMeta(, c8b8caf9-7d05-45e6-9068-9d835cc11ee9, 26, Finished, Available, Finished)

AnalysisException: [SCHEMA_NOT_FOUND] The schema `bronze` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.